# VAR India

Vector auto-regressive models - VAR, VMA, VARIMA on Covid-19 Cases. For all models, the first-differenced dataset is used (I = 1).

## [Setup](#setup)
1. [Imports](#imports)
2. [Results table](#results_init)
3. [Ingestion](#ingestion)
4. [Plotting](#plotting_init)
5. [Statistical tests](#stattests1)
    1. [Johansen co-integration test](#jci_init)
    2. [Augmented DF Test](#adf_init)
6. [Differencing](#diff_init)
7. [Train test split](#traintest_init)

## Long-Term Forecasting

### [VAR Model](#var_model)
1. [Find order p of VAR](#var_p)
2. [VAR(1) Model](#var1)
3. [Plots of first differenced predictions](#diff_plot_var)
4. [Undifferencing and plotting](#undiff_var)
5. [Store metrics - MAE, MAPE, MSE](#store_var)
6. [Plot Train, Test, Forecast](#plot_final_var)

    
### [VMA Model](#vma_model)
1. [Find order q of VMA](#vma_q)
2. [VMA(1) Model](#vma1)
3. [Plots of first differenced predictions](#diff_plot_vma)
4. [Undifferencing and plotting](#undiff_vma)
5. [Store metrics - MAE, MAPE, MSE](#store_vma)
6. [Plot Train, Test, Forecast](#plot_final_vma)


### [VARMA Model](#varma_model)
1. [Find order (p, q) of VARMA](#varma_pq)
2. [VARMA(1, 1) Model](#varma11)
3. [Plots of first differenced predictions](#diff_plot_varma)
4. [Undifferencing and plotting](#undiff_varma)
5. [Store metrics - MAE, MAPE, MSE](#store_varma)
6. [Plot Train, Test, Forecast](#plot_final_varma)

## [Short-Term/Rolling Forecasting](#shortterm)

1. [VAR(1)](#var_roll)
2. [VMA(1)](#vma_roll)
3. [VARMA(1,1)](#varma_roll)

## [Final Metrics](#final_results)

<a name=setup></a>

# Setup

<a name=imports></a>
## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

<a name=results_init></a>
## Results table

In [ ]:
results_columns = ['model', 'mse', 'mape', 'mae']
results_table = pd.DataFrame(columns=results_columns)

<a name=ingestion></a>

## Ingestion

In [ ]:
daily_cases_india = pd.read_csv('../../cleaned_datasets/india/daily_cases_india.csv', parse_dates=['Date'])
daily_vacc_india = pd.read_csv('../../cleaned_datasets/india/daily_vacc_india.csv', parse_dates=['date'])

In [ ]:
daily_cases_india.dtypes

In [ ]:
daily_cases_india

In [ ]:
daily_vacc_india.dtypes

In [ ]:
daily_vacc_india

In [ ]:
cases_vacc = daily_cases_india.merge(daily_vacc_india, how='outer', left_on='Date', right_on='date')
cases_vacc = cases_vacc[["Date", "Confirmed", "Total_Doses"]]
cases_vacc

In [ ]:
cases_vacc.fillna(0, inplace=True)
indexed = cases_vacc.set_index('Date')
indexed

<a name=plotting_init></a>
## Plot initial data

In [ ]:
def plot_2col_subplots(indexed):
    
    if type(indexed) == pd.DataFrame:
        nrows = int(len(indexed.columns)/2)
    else:
        nrows = 1
        
    fig, axes = plt.subplots(nrows=nrows, ncols=2, dpi=120, figsize=(8,4))
    for i, ax in enumerate(axes.flatten()):
        
        if type(indexed) == list:
            # fig, ax = plt.subplots()
            # fig.set_size_inches(8, 8)

            ax.plot(indexed[0][indexed[0].columns[i]], color='blue', label = 'Train')
            ax.plot(indexed[1][indexed[1].columns[i]], color='red', label = 'Test')
            ax.legend(loc = 'best')
            # Decorations
            ax.set_title(indexed[0].columns[i])
        else:    
            data = indexed[indexed.columns[i]]
            ax.plot(data, color='blue', linewidth=1)
            # Decorations
            ax.set_title(indexed.columns[i])
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.spines["top"].set_alpha(0)
        ax.tick_params(labelsize=6)
    
    plt.tight_layout()
    
plot_2col_subplots(indexed)

<a name=stattests1></a>
## Statistical tests

<a name=jci_init></a>

### Johansen co-integration test

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(indexed)

From JCT, the two time series are not correlated.

<a name=adf_init></a>

### Augmented DF Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def run_dicky_fuller(ts):
  '''Function to run Augmented Dicky Fuller test on the passed time series and report the statistics from the test'''
  print("Observations of Dickey-fuller test")
  dftest = adfuller(ts,autolag='AIC')
  dfoutput=pd.Series(dftest[0:4],index=['Test Statistic','p-value','#lags used','number of observations used'])

  for key,value in dftest[4].items():
      dfoutput['critical value (%s)'%key]= value
  print(dfoutput)


# ADF Test on each column
for name, column in indexed.iteritems():
    run_dicky_fuller(column)
    print('\n')

The TS is not stationary

<a name=diff_init></a>
## Differencing

## First order differencing

In [ ]:
df_diff_1 = indexed.diff().dropna()
df_diff_1

In [ ]:
plot_2col_subplots(df_diff_1)


In [ ]:
# ADF Test on each column
for name, column in df_diff_1.iteritems():
    run_dicky_fuller(column)
    print('\n')

In [ ]:
cointegration_test(df_diff_1)

From JCT, the two time series are correlated.

<a name=traintest_init></a>
## Train-test split

### Train-test split - first order differenced

In [ ]:
percent_90 = int(len(df_diff_1)*0.9)

train = df_diff_1.iloc[:percent_90].dropna()
test = df_diff_1.iloc[percent_90:]

plot_2col_subplots([train, test])

<a name=var_model></a>

# VAR Model

<a name=var_p></a>
## Finding the order p for VAR(p)

Using PACF plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

pacf_var_confirmed = plot_pacf(df_diff_1['Confirmed'], lags=25)

In [ ]:
def MAPE(Y_actual, Y_Predicted, title=None):
    mask = Y_actual != 0
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual)[mask])*100
    if title:
        print(f"MAPE of {title} is {mape[mape.index[0]]}%")
    return mape

In [ ]:
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

# evaluate an VARMA model for a given order (p,d,q)
def evaluate_varma_model(train, test, varma_order, column):
    
    model = VARMAX(train, order=varma_order)
    model_fit = model.fit()
    yhat = model_fit.forecast(len(test))
    mse = mean_squared_error(test[column], yhat[column])
    mape = MAPE(test[column], yhat[column])
    mae = mean_absolute_error(test[column], yhat[column])
    return mse, mape, mae

In [ ]:
# Grid search

# evaluate combinations of p and q values for an VARMA model
def evaluate_models(train, test, p_values, q_values, column):
    #dataset = dataset.astype('float32')
    best_score, best_cfg = {'mse': float("inf")}, None
    
    for p in p_values:
        for q in q_values:
            order = (p,q)
            try:
                mse, mape, mae = evaluate_varma_model(train, test, order, column)
                
                if mse < best_score['mse']:
                    best_score['mse'], best_cfg = mse, order
                
                print('VARMA%s MSE=%.3f, MAPE=%.3f, MAE=%.3f' % (order,mse,mape,mae))
            except:
                continue
    print()
    print('Best VARMA%s MSE=%.3f' % (best_cfg, best_score['mse']))

In [ ]:
evaluate_models(train, test, [1, 9], [0], column='Confirmed')

From trial and error, VAR(9) yields negative values while VAR(1) does not. Therefore, VAR(1) chosen.

<a name=var1></a>
## VAR(1) Model

In [ ]:
model = VARMAX(train, order=(1,0))
model_fit = model.fit()
forecasted = model_fit.forecast(len(test))

df_forecast = pd.DataFrame(forecasted, index=test.index)
df_forecast.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf = df_forecast.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc = df_forecast.join(test)[['Total_Doses', 'Total_Doses_forecast']]

# conf = model_fit.conf_int(alpha=0.05)

In [ ]:
model_fit.summary()

<a name=diff_plot_var></a>

## Plots of Predictions (Differenced)

In [ ]:
def plot_fore_test(test, fore, title):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(12, 8)

    ax.plot(test, color='blue', label='Test')
    ax.plot(fore, color='red', label='Forecast')
    ax.legend(loc='best')
    plt.title(title)
    plt.show()

In [ ]:
# plot_fore_test(test[['Confirmed']], forecasted[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], forecasted[['Total_Doses']], title='Diffed Daily vaccinations')

<a name=undiff_var></a>

## Un-differencing and Plotting

In [ ]:
def invert_transformation(diffed, original):
    """Revert back the differencing to get the forecast to original scale."""
    df_copy = original.copy()
    
    columns = diffed.columns
    
    for col in columns:
        # Add corresponding diff column
        df_copy[f'undiff_{col}'] = original[col]
        df_copy[f'undiff_{col}'][1:] = diffed[col]
        df_copy[f'undiff_{col}'] = df_copy[f'undiff_{col}'].cumsum()
    
        df_copy.drop(col, axis=1, inplace=True)
        
    return df_copy

In [ ]:
# Check if un-diffed df_diff_1 is the same as original

#inverted = invert_transformation(df_diff_1, indexed)
#(inverted[['undiff_Confirmed']] - indexed[['Confirmed']]).isna().sum()

In [ ]:
# Un-diff the test dataset

start_index = indexed.index.get_loc(test.index[0])-1
test_original = invert_transformation(test, indexed.iloc[start_index:])

#test_original

In [ ]:
renamed_df = df_forecast.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index = indexed.index.get_loc(renamed_df.index[0])-1
fore_original = invert_transformation(renamed_df, indexed.iloc[start_index:])

In [ ]:
end_index = indexed.index.get_loc(fore_original.index[0])
train_original = indexed[:end_index]
# plot_2col_subplots([train_original, fore_original])

<a name=store_var></a>

## Store metrics

In [ ]:
results_table = results_table.append({
    'model': 'VAR(1)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed'])
}, ignore_index=True)

results_table

<a name=plot_final_var></a>

## Plot Train, Test, Forecast

In [ ]:
def plot_train_test_fore(train, test, fore, conf=None, title='Forecast vs Actuals', ylabel='', xlabel='Date', figpath=None):
    
    # Confidence of cases
    if conf is not None:
        lower_series = pd.Series(conf[:, 0], index=val.index)
        upper_series = pd.Series(conf[:, 1], index=val.index)
        plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
        
    
    plt.figure(figsize=(10,5), dpi=100)
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fore, label='forecast')
    fig = plt.gcf()
    
    plt.xlabel=xlabel
    plt.ylabel=ylabel

    plt.title(title)
    plt.legend(loc='upper left', fontsize=8)
    plt.show()
    
    
    if figpath is not None:
        fig.savefig(figpath, format='eps', bbox_inches='tight')

In [ ]:
# Plot of daily cases
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original[['undiff_Confirmed']], title='Daily cases')


Clearly, a VAR model is not good enough to make predictions

In [ ]:
# Plot of daily doses
#plot_train_test_fore(train_original.Total_Doses, test_original[['undiff_Total_Doses']], fore_original[['undiff_Total_Doses']], title='Daily doses', figpath='../../figures/v_ar/india_vacc.eps')


<a name=vma_model></a>

# VMA Model

<a name=vma_q></a>
## Find order q of VMA

In [ ]:
acf_varma_confirmed = plot_acf(train['Confirmed'], lags=25)

In [ ]:
evaluate_models(train, test, [0], [1, 7], column='Confirmed')

<a name=vma1></a>

## VMA(1)

In [ ]:
model_ma = VARMAX(train, order=(0,1))
model_fit_ma = model_ma.fit()
forecasted_ma = model_fit_ma.forecast(len(test))

df_forecast_ma = pd.DataFrame(forecasted_ma, index=test.index)
df_forecast_ma.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf_ma = df_forecast_ma.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc_ma = df_forecast_ma.join(test)[['Total_Doses', 'Total_Doses_forecast']]

# conf = model_fit.conf_int(alpha=0.05)

In [ ]:
model_fit_ma.summary()

<a name=diff_plot_vma></a>

## Plots of Predictions (Differenced)

In [ ]:
plot_fore_test(test[['Confirmed']], forecasted_ma[['Confirmed']], title='Diffed Daily cases')

In [ ]:
plot_fore_test(test[['Total_Doses']], forecasted_ma[['Total_Doses']], title='Diffed Daily vaccinations')

<a name=undiff_vma></a>

## Undifferencing and plotting

In [ ]:
renamed_df_ma = df_forecast_ma.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index_ma = indexed.index.get_loc(renamed_df_ma.index[0])-1
fore_original_ma = invert_transformation(renamed_df_ma, indexed.iloc[start_index_ma:])

In [ ]:
fore_original_ma

<a name=store_vma></a>

## Store metrics

In [ ]:
results_table = results_table.append({
    'model': 'VMA(1)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_ma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_ma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_ma['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
# end_index = indexed.index.get_loc(fore_original_ma.index[0])
# train_original_ma = indexed[:end_index]
# #plot_subplots([train_original, fore_original_ma])

<a name=plot_final_vma></a>

## Plot Train, Test, Forecast

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_ma[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/v_ar/india_cases.eps')


<a name=varma_model></a>

# VARMA

<a name=varma_pq></a>

## Find order (p, q) of VARMA

PACF/ACF

In [ ]:
pacf_varma_confirmed = plot_pacf(train['Confirmed'], lags=25)
acf_varma_confirmed = plot_acf(train['Confirmed'], lags=25)

VARIMA(1, 1), VARIMA(9, 1), VARIMA(1, 7), VARIMA(9, 7)

In [ ]:
evaluate_models(train, test, [1, 9], [1, 7], column='Confirmed')

<a name=varma11></a>

## VARMA(1, 1)

In [ ]:
model = VARMAX(train, order=(1,1))
model_fit = model.fit()
yhat = model_fit.forecast(len(test))
#yhat

<a name=diff_plot_varma></a>

## Plots of Predictions (Differenced)

In [ ]:
plot_fore_test(test[['Confirmed']], yhat[['Confirmed']], title='Diffed Daily cases')

In [ ]:
plot_fore_test(test[['Total_Doses']], yhat[['Total_Doses']], title='Diffed Daily Doses')

<a name=undiff_varma></a>

## Undifferencing and plotting

In [ ]:
# Un-diff the test dataset

fore_original_3 = invert_transformation(yhat, indexed.iloc[start_index:])
fore_original_3

In [ ]:
#plot_fore_test(test_original[['undiff_Confirmed']], fore_original_3[['undiff_Confirmed']], title='Daily cases')

In [ ]:
#plot_fore_test(test_original[['undiff_Total_Doses']], fore_original_3[['undiff_Total_Doses']], title='Daily Doses')

In [ ]:
plot_2col_subplots([train_original, fore_original_3])

<a name=store_varma></a>

## Store metrics

In [ ]:
results_table = results_table.append({
    'model': 'VARMA(1,1)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_3['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_3['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_3['undiff_Confirmed'])
}, ignore_index=True)

results_table

<a name=plot_final_varma></a>

## Plot Train, Test, Forecast

In [ ]:
# Plot of daily cases
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_3[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/varma/india_cases.eps')


In [ ]:
# Plot of daily doses
# plot_train_test_fore(train_original.Total_Doses, test_original[['undiff_Total_Doses']], fore_original_3[['undiff_Total_Doses']], title='Daily cases', figpath='../../figures/varma/india_vacc.eps')

<a name=shortterm></a>

# Rolling forecasts

In [ ]:
history = train.copy()
predicted = pd.DataFrame(columns=[
    'VAR_Confirmed', 
    'VAR_Total_Doses', 
    'VMA_Confirmed', 
    'VMA_Total_Doses', 
    'VARMA_Confirmed', 
    'VARMA_Total_Doses'
], index=test.index)


# predicted

for t in range(len(test)):
    # 3 models
    var = VARMAX(history, order=(1,0))
    vma = VARMAX(history, order=(0,1))
    varma = VARMAX(history, order=(1,1))
    
    # 3 fitted models
    var_fit = var.fit()
    vma_fit = vma.fit()
    varma_fit = varma.fit()
    
    # Predictions
    yhat_var = var_fit.forecast()
    yhat_vma = vma_fit.forecast()
    yhat_varma = varma_fit.forecast()
    
    # Next index to insert
    newindex = history.index[-1] + pd.to_timedelta(1, 'D')
    
    # Confirmed cases
    predicted.loc[newindex]['VAR_Confirmed'] = yhat_var['Confirmed'].values[0]
    predicted.loc[newindex]['VMA_Confirmed'] = yhat_vma['Confirmed'].values[0]
    predicted.loc[newindex]['VARMA_Confirmed'] = yhat_varma['Confirmed'].values[0]
    
    # Total doses
    predicted.loc[newindex]['VAR_Total_Doses'] = yhat_var['Total_Doses'].values[0]
    predicted.loc[newindex]['VMA_Total_Doses'] = yhat_vma['Total_Doses'].values[0]
    predicted.loc[newindex]['VARMA_Total_Doses'] = yhat_varma['Total_Doses'].values[0]
    
    history = history.append(test.iloc[t])
    # history[newindex]['Total_Doses'] = test.iloc[t]['Total_Doses']
    
    # print('predicted =', yhat.values, ' ; actual =', test.iloc[t].values)

In [ ]:
predicted

<a name=var_roll></a>

## VAR(1)

In [ ]:
plot_subplots([test, predicted[['VAR_Confirmed', 'VAR_Total_Doses']]])

In [ ]:
# Un-diffing

pred_var = predicted[['VAR_Confirmed', 'VAR_Total_Doses']].rename(columns={'VAR_Confirmed': 'Confirmed', 'VAR_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_4_var = invert_transformation(pred_var, indexed.iloc[start_index:])

In [ ]:
# fore_original_4_var

In [ ]:
results_table = results_table.append({
    'model': 'VAR(1) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_4_var['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_4_var['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_4_var['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_4_var[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/varma/india_cases.eps')


<a name=vma_roll></a>

## VMA(1)

In [ ]:
plot_subplots([test, predicted[['VMA_Confirmed', 'VMA_Total_Doses']]])

In [ ]:
# Un-diffing

pred_vma = predicted[['VMA_Confirmed', 'VMA_Total_Doses']].rename(columns={'VMA_Confirmed': 'Confirmed', 'VMA_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_4_vma = invert_transformation(pred_vma, indexed.iloc[start_index:])

In [ ]:
# fore_original_4_vma

In [ ]:
results_table = results_table.append({
    'model': 'VMA(1) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_4_vma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_4_vma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_4_vma['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_4_vma[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/varma/india_cases_vma.eps')



<a name=varma_roll></a>

## VARMA(1,1)

In [ ]:
plot_subplots([test, predicted[['VARMA_Confirmed', 'VARMA_Total_Doses']]])

In [ ]:
# Un-diffing

pred_varma = predicted[['VARMA_Confirmed', 'VARMA_Total_Doses']].rename(columns={'VARMA_Confirmed': 'Confirmed', 'VARMA_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_4_varma = invert_transformation(pred_varma, indexed.iloc[start_index:])

In [ ]:
# fore_original_4_varma

In [ ]:
results_table = results_table.append({
    'model': 'VARMA(1) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_4_varma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_4_varma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_4_varma['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_4_varma[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/varma/india_cases_varma.eps')


<a name=final_results></a>

# Final Results

In [ ]:
results_table